# Zoom Polarization Analysis

## Introduction

In [ ]:
import sciline
import scipp as sc
from ess import polarization as pol
from ess import isissans as isis
from ess.sans.types import *

In [ ]:
sans_workflow = isis.zoom.ZoomWorkflow()
sans_workflow.set_param_series(PixelMaskFilename, [])

In [ ]:
from pathlib import Path

base = Path(
    '~/instruments/polarization/Polarized_data_ZOOM/full_polarized_TOF-SANS_example_data_on_glassy_carbon'
)
data_folder = base / 'Long_3He_run'
data_folder = base / 'GC/3He_GC'
# Runs with analyzer at 4 different times
cell_runs = [data_folder / f'ZOOM00022{run}.nxs' for run in [710, 712, 714, 716]]
empty_run = data_folder / 'ZOOM00034787.nxs'
cell_runs
depolarized_run = data_folder / 'ZOOM00022718.nxs'

## Setup SANS workflow for computing transmission fraction

In [ ]:
#new:
from ess.polarization.zoom import ZoomTransmissionFractionWorkflow

sans_workflow = ZoomTransmissionFractionWorkflow()
sans_workflow[Filename[EmptyBeamRun]] = str(empty_run)
sans_workflow[WavelengthBins] = sc.geomspace(
    'wavelength', start=1.75, stop=16.5, num=141, unit='Å'
)
sans_workflow[isis.mantidio.Period] = 0

In [ ]:
#Old:
from ess.polarization.zoom import load_histogrammed_run, get_incident, get_transmission

sans_workflow[Filename[SampleRun]] = str(cell_runs[0])
sans_workflow[Filename[TransmissionRun[SampleRun]]] = str(cell_runs[0])
sans_workflow[Filename[EmptyBeamRun]] = str(empty_run)
sans_workflow[NeXusMonitorName[Incident]] = 'monitor3'
sans_workflow[NeXusMonitorName[Transmission]] = 'monitor5'
sans_workflow.insert(load_histogrammed_run)
sans_workflow.insert(get_incident)
sans_workflow.insert(get_transmission)
sans_workflow[WavelengthBins] = sc.geomspace(
    'wavelength', start=1.75, stop=16.5, num=141, unit='angstrom'
)
sans_workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound
sans_workflow[isis.mantidio.Period] = 0
dg = sans_workflow.compute(isis.mantidio.LoadedFileContents[TransmissionRun[SampleRun]])
dg['data'].plot()

## Inspect data for one of the runs with analyzer cell

In [ ]:
# Load only first run
sans_workflow.set_param_series(Filename[TransmissionRun[SampleRun]], cell_runs[:1])
loaded = sans_workflow.compute(
    sciline.Series[
        Filename[TransmissionRun[SampleRun]],
        isis.mantidio.LoadedFileContents[TransmissionRun[SampleRun]],
    ]
)
first_run = list(loaded.values())[0]
sc.DataGroup(sc.collapse(first_run['data'], keep='tof')).plot()

In [ ]:
first_run

In [ ]:
#Old:
sans_workflow.visualize(TransmissionFraction[SampleRun], graph_attr={'rankdir': 'LR'})
transmission_fraction = sans_workflow.compute(TransmissionFraction[SampleRun])
transmission_fraction.plot()
# display(sans_workflow.compute(TofMonitor[SampleRun, Incident]).plot())
# display(sans_workflow.compute(TofMonitor[SampleRun, Transmission]).plot())
# display(sans_workflow.compute(TofMonitor[EmptyBeamRun, Incident]).hist().plot())
# display(sans_workflow.compute(TofMonitor[EmptyBeamRun, Transmission]).hist().plot())

sans_workflow[Filename[TransmissionRun[SampleRun]]] = str(depolarized_run)
depolarized_transmission_fraction = sans_workflow.compute(
    TransmissionFraction[SampleRun]
)
depolarized_transmission_fraction.plot()

We can load the combined time-dependent incident and transmission monitors.
Note that the last run is the depolarized run:

In [ ]:
sans_workflow.set_param_series(Filename[TransmissionRun[SampleRun]], cell_runs)
mons = sans_workflow.compute(
    (
        RawMonitor[TransmissionRun[SampleRun], Incident],
        RawMonitor[TransmissionRun[SampleRun], Transmission],
    )
)
mons = sc.DataGroup(
    incident=mons[RawMonitor[TransmissionRun[SampleRun], Incident]],
    transmission=mons[RawMonitor[TransmissionRun[SampleRun], Transmission]],
)
display(sc.DataGroup(sc.collapse(mons['incident'], keep='tof')).plot())
display(sc.DataGroup(sc.collapse(mons['transmission'], keep='tof')).plot())

The task graph for computing the transmission fraction is:

In [ ]:
sans_workflow.visualize(TransmissionFraction[SampleRun], graph_attr={'rankdir': 'LR'})

## Compute transmission fractions

There are multiple files which together define the time-dependence of the analyzer cell transmission.
Note that as before the final run (time) is the depolarized run:

In [ ]:
raw_transmission = sans_workflow.compute(TransmissionFraction[SampleRun])

In [ ]:
# TODO Which wavelength bounds should be used?
wav_min = 2.2 * sc.Unit('angstrom')
wav_max = 5.8 * sc.Unit('angstrom')
transmission_truncated = raw_transmission['wavelength', wav_min:wav_max]
transmission_depolarized = transmission_truncated['time', -1].copy()
transmission = transmission_truncated['time', :-1].copy()

We can plot the computed transmission fractions:

In [ ]:
trans = sc.DataGroup(
    {
        f"{t.coords['time']:c}": t
        for t in sc.collapse(transmission, keep='wavelength').values()
    }
)
trans[f'depolarized ({transmission_depolarized.coords["time"]:c})'] = (
    transmission_depolarized
)
display(trans.plot())

In [ ]:
# Sanity check: Where can cosh yield values that can be fitted?
transmission_empty_glass = 0.9 * sc.Unit('dimensionless')
wavelength = sc.midpoints(transmission.coords['wavelength'])
opacity0 = 0.8797823016804095 * sc.Unit('1/angstrom')
(
    sc.acosh(transmission * sc.exp(opacity0 * wavelength) / transmission_empty_glass)
    / (opacity0 * wavelength)
).plot()

We can now setup the polarization analysis workflow.
The previously computed transmission fractions are used as workflow inputs:

In [ ]:
#new:
pol_workflow = pol.he3.He3CellWorkflow(in_situ=False, incoming_polarized=True)
pol_workflow[pol.he3.He3CellTransmissionFraction[pol.Analyzer, pol.Polarized]] = (
    transmission
)
pol_workflow[pol.he3.He3CellTransmissionFraction[pol.Analyzer, pol.Depolarized]] = (
    transmission_depolarized
)

# When in_situ=False, these params are used as starting guess for the fit
pol_workflow[pol.he3.He3CellLength[pol.Analyzer]] = 0.1 * sc.Unit('m')
pol_workflow[pol.he3.He3CellPressure[pol.Analyzer]] = 1.0 * sc.Unit('bar')
pol_workflow[pol.he3.He3CellTemperature[pol.Analyzer]] = 300.0 * sc.Unit('K')

pol_workflow[pol.he3.He3TransmissionEmptyGlass[pol.Analyzer]] = transmission_empty_glass
pol_workflow.visualize(
    pol.he3.He3TransmissionFunction[pol.Analyzer], graph_attr={'rankdir': 'LR'}
)

In [ ]:
#Old:
pol_workflow = pol.he3.He3CellWorkflow(in_situ=False)
pol_workflow.insert(pol.he3.get_he3_transmission_from_fit_to_direct_beam_from_polarized_incoming_beam)
#Note:  If line 2 is in, we automatically use the function for incoming polarized beam instead of incming unpolarized (what Simon has written before)
#       If an incoming unpolarized beam is used, comment line 2!
pol_workflow[
    pol.he3.He3CellTransmissionFraction[pol.Analyzer, pol.Polarized]
] = transmission_fraction
pol_workflow[
    pol.he3.He3CellTransmissionFraction[pol.Analyzer, pol.Depolarized]
] = depolarized_transmission_fraction

# When in_situ=False, these params are used as starting guess for the fit
pol_workflow[pol.he3.He3CellLength[pol.Analyzer]] = 0.1 * sc.Unit('m')
pol_workflow[pol.he3.He3CellPressure[pol.Analyzer]] = 1.0 * sc.Unit('bar')
pol_workflow[pol.he3.He3CellTemperature[pol.Analyzer]] = 300.0 * sc.Unit('K')

pol_workflow[pol.he3.He3TransmissionEmptyGlass[pol.Analyzer]] = transmission_empty_glass
pol_workflow.visualize(
    pol.he3.He3TransmissionFunction[pol.Analyzer], graph_attr={'rankdir': 'LR'}
)

The workflow can compute the transmission function:

In [ ]:
func = pol_workflow.compute(pol.he3.He3TransmissionFunction[pol.Analyzer])

We can evaluate this transmission function at desired time and wavelength points:

In [ ]:
wavelength = sc.linspace('wavelength', start=1.75, stop=16.5, num=141, unit='angstrom')
time = sc.linspace('time', start=0, stop=100000, num=100, unit='s')
display(func.opacity_function(wavelength=wavelength).plot())
display(func.polarization_function(time=time).plot())
display(func(wavelength=wavelength, time=time, plus_minus='plus').plot(norm='log'))
display(func(wavelength=wavelength, time=time, plus_minus='minus').plot(norm='log'))

In [ ]:
func.opacity_function.opacity0

In [ ]:
func.polarization_function.C

In [ ]:
func.polarization_function.T1